In [9]:
class numerics:
    class shift():
        def __init__(self, plus, mnus):
            self.plus = plus
            self.mnus = mnus

        def __radd__(self, arg):
            return type(arg)(
                arg.start + self.plus,
                arg.stop + self.plus
            )

        def __rsub__(self, arg):
            return type(arg)(
                arg.start - self.mnus,
                arg.stop - self.mnus
            )
        
        def __rmod__(self, arg):
            return type(arg)(
                arg.start - self.mnus,
                arg.stop + self.plus
            )
        
        def __rmul__(self, arg):
            return type(self)(
                arg * self.plus,
                arg * self.mnus
            )

    def __init__(self):
        self.one = self.shift(1,1)
        self.hlf = self.shift(0,1)
    
    def F(self, psi_l, psi_r, GC):
        return np.maximum(0, GC) * psi_l + np.minimum(0, GC) * psi_r
    
    def upwind(self, psi, GC, G, n, i):
        psi[n+1][i] = psi[n][i] - (
            self.F(psi[n][i],          psi[n][i+self.one], GC[i+self.hlf]) - 
            self.F(psi[n][i-self.one], psi[n][i],          GC[i-self.hlf])
        ) / G[i]
    
    def GC_antidiff(self, opts, psi, GC, G, ih):
        h = self.hlf
        o = self.one
        i = ih + h # TODO !!! (dziala, rozumiemy, ale brzydkie)
        
        result = (
            GC[i+h] - GC[i+h]**2 
            / 
            (.5 *(G[i+o] + G[i]))  
        ) * (
            (psi[i+o] - psi[i]) 
            /
            (psi[i+o] + psi[i])
        )
        
        if opts["dfl"]:
            result -= .5 * (GC[i+h]) / (G[i+o] + G[i]) * (GC[i+o+h] - GC[i-h])
        
        return result